### Import tensorflow and list available devices

In [1]:
import tensorflow as tf
import platform

# Print TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

# Print information about all physical devices
physical_devices = tf.config.list_physical_devices()
print("Physical devices:")
for device in physical_devices:
    print(device)

# Print information about the CPU
print(f"Platform: {platform.platform()}")
print(f"Processor: {platform.processor()}")



2024-02-03 16:56:06.074416: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.12.0
Physical devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Platform: macOS-10.16-x86_64-i386-64bit
Processor: i386


### Download the data and split it into training and testing sets

In [2]:
import tensorflow_datasets as tfds
from pathlib import Path

BATCH_SIZE = 32
NUM_CLASSES = 3
# dataset will be placed here
data_dir = '../datasets/'
Path('../datasets/').mkdir(parents=True, exist_ok=True)

(ds_train, ds_test), ds_info = tfds.load(
    "beans",
    split=["train", "test"],
    batch_size=BATCH_SIZE,
    with_info=True,
    as_supervised=True,
    shuffle_files=True,
    data_dir=data_dir
)
ds_train = ds_train.map(lambda image, label: (tf.image.resize_with_crop_or_pad(image, 224, 224),
                        tf.cast(label, tf.int32)))
ds_test = ds_test.map(lambda image, label: (tf.image.resize_with_crop_or_pad(image, 224, 224),
                        tf.cast(label, tf.int32)))

print(f"Batch shape: {list(ds_train.element_spec[0].shape)}")
print(f"Num train images: {len(ds_train) * BATCH_SIZE}")
print(f"Num test images: {len(ds_test) * BATCH_SIZE}")

/Users/eriksommer/miniconda3/envs/tensorflow-metal/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-03 16:56:15.981625: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 100%|██████████| 3/3 [00:20<00:00,  6.74s/ url]
2024-02-03 16:56:41.236431: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-03 16:56:41.236491: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Dataset beans downloaded and prepared to ../datasets/beans/0.1.0. Subsequent calls will reuse this data.
Batch shape: [None, 224, 224, 3]
Num train images: 1056
Num test images: 128


### Create and compile the model

In [3]:
model = tf.keras.applications.resnet50.ResNet50(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=NUM_CLASSES
)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])

### Running training on CPU for comparison

In [6]:
time = tf.timestamp()
with tf.device('CPU'):
    model.fit(
        ds_train,
        epochs=2,
        batch_size=BATCH_SIZE,
    )
print(f"CPU training time: {tf.timestamp() - time} seconds")

Epoch 1/2
33/33 [==============================] - 251s 8s/step - loss: 0.8388 - accuracy: 0.6460
Epoch 2/2
33/33 [==============================] - 250s 8s/step - loss: 0.8234 - accuracy: 0.6867
CPU training time: 501.7559278011322 seconds


### Running same training on GPU

In [5]:
time = tf.timestamp()
with tf.device('GPU'):
    model.fit(
        ds_train,
        epochs=2,
        batch_size=BATCH_SIZE,
    )
print(f"GPU training time: {tf.timestamp() - time} seconds")

Epoch 1/2
33/33 [==============================] - 20s 591ms/step - loss: 0.7336 - accuracy: 0.6973
Epoch 2/2
33/33 [==============================] - 20s 594ms/step - loss: 0.6847 - accuracy: 0.7263
GPU training time: 40.497875928878784 seconds
